In [12]:
import pandas as pd
import numpy as np
import pandas_profiling
import re

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from tpot import TPOTRegressor
from auto_ml import Predictor
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')

In [48]:
df = pd.read_csv('client_training_cleaned_1.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [49]:
df2 = pd.get_dummies(df, prefix_sep='_', drop_first=True)

X = df2.drop(['job_performance'], axis=1)
y = df2['job_performance']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
xg = xgb.XGBRegressor(n_estimators=100, max_depth=3)
xg.fit(X_train, y_train)

y_pred_xg = xg.predict(X_test)

# Evaluation metrics
mse_xg = mean_squared_error(y_test, y_pred_xg)
r2_xg = r2_score(y_test, y_pred_xg)
print('MSE:\n{}\n\nr2_score:\n{}'.format(mse_xg, r2_xg))

In [ ]:
rf = RandomForestRegressor(random_state = 0)
rf.fit(X_train, y_train)
# df_imp_3 = pd.DataFrame(rf_3.feature_importances_, 
#             index = X_train.columns, columns = ['importance']).sort_values('importance', ascending=False)

# Evaluation
y_pred_rf = rf.predict(X_test)

# Evaluation metrics
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print('MSE:\n{}\n\nr2_score:\n{}'.format(mse_rf, r2_rf))

In [ ]:
categorical_clms = df.select_dtypes(include=['O']).columns
print("Number of categorical columns: {}".format(len(categorical_clms)))

numerical_clms = df.select_dtypes(include=['int64', 'float64']).columns
print("Number of numerical columns: {}".format(len(numerical_clms)))

df_cat = pd.DataFrame(df, columns=categorical_clms)
df_num = pd.DataFrame(df, columns=numerical_clms)

In [ ]:
from fancyimpute import KNN

df_org = pd.read_csv('client-trainingset-1561457457-252.csv')

df_num_knn = df_org[numerical_clms]

df_num_knn_filled = KNN(k=3).fit_transform(df_num_knn)

In [ ]:
df_num.columns.values

array(['yrsqual', 'earnhr', 'v272', 'ictwork', 'planning', 'readhome',
       'v207', 'v252', 'v242', 'influence', 'yrsget', 'row', 'v210',
       'writwork', 'learnatwork', 'v136', 'v33', 'earnmth', 'nfehrsnjr',
       'icthome', 'readwork', 'nfehrsjr', 'taskdisc', 'v239', 'isco1c',
       'v105', 'writhome', 'readytolearn', 'v202', 'v104', 'v133',
       'nfehrs', 'v206', 'v224', 'earnhrppp', 'v187', 'leavedu', 'v22',
       'v154', 'v135', 'job_performance'], dtype=object)

In [ ]:
df_numeric = pd.DataFrame(df_num_knn_filled, columns=['yrsqual', 'earnhr', 'v272', 'ictwork', 'planning', 'readhome',
       'v207', 'v252', 'v242', 'influence', 'yrsget', 'row', 'v210',
       'writwork', 'learnatwork', 'v136', 'v33', 'earnmth', 'nfehrsnjr',
       'icthome', 'readwork', 'nfehrsjr', 'taskdisc', 'v239', 'isco1c',
       'v105', 'writhome', 'readytolearn', 'v202', 'v104', 'v133',
       'nfehrs', 'v206', 'v224', 'earnhrppp', 'v187', 'leavedu', 'v22',
       'v154', 'v135', 'job_performance'])

In [ ]:
df4 = pd.get_dummies(df3, prefix_sep='_', drop_first=True)

X_2 = df4.drop(['job_performance'], axis=1)
y_2 = df4['job_performance']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.3, random_state=0)

rf_2 = RandomForestRegressor(random_state = 0)
rf_2.fit(X_train_2, y_train_2)
# df_imp_3 = pd.DataFrame(rf_3.feature_importances_, 
#             index = X_train.columns, columns = ['importance']).sort_values('importance', ascending=False)

# Evaluation
y_pred_rf_2 = rf_2.predict(X_test_2)

# Evaluation metrics
mse_rf_2 = mean_squared_error(y_test_2, y_pred_rf_2)
r2_rf_2 = r2_score(y_test, y_pred_rf)
print('MSE:\n{}\n\nr2_score:\n{}'.format(mse_rf_2, r2_rf_2))

# TPOT

In [4]:
tpot = TPOTRegressor(generations=5, population_size=50, verbosity=2)
tpot.fit(X_train, y_train)

y_pred_tp = tpot.predict(X_test)

mse_tp = mean_squared_error(y_test, y_pred_tp)
r2_tp = r2_score(y_test, y_pred_tp)
print('MSE:\n{}\n\nr2_score:\n{}'.format(mse_tp, r2_tp))

Generation 1 - Current best internal CV score: -45738.190270667794
Generation 2 - Current best internal CV score: -45738.190270667794
Generation 3 - Current best internal CV score: -43666.9964926823
Generation 4 - Current best internal CV score: -43666.9964926823
Generation 5 - Current best internal CV score: -37854.37598091162

Best pipeline: RandomForestRegressor(input_matrix, bootstrap=False, max_features=0.15000000000000002, min_samples_leaf=3, min_samples_split=11, n_estimators=100)
MSE:
33358.83344019907

r2_score:
0.8185459963530592


# RANDOM FOREST

In [12]:
rf_3 = RandomForestRegressor(bootstrap=False, max_features=0.15000000000000002, min_samples_leaf=3,
                             min_samples_split=11, n_estimators=100)
rf_3.fit(X_train, y_train)
# df_imp_3 = pd.DataFrame(rf_3.feature_importances_, 
#             index = X_train.columns, columns = ['importance']).sort_values('importance', ascending=False)

# Evaluation
y_pred_rf_3 = rf_3.predict(X_test)

# Evaluation metrics
mse_rf_3 = mean_squared_error(y_test, y_pred_rf_3)
r2_rf_3 = r2_score(y_test, y_pred_rf_3)
print('MSE:\n{}\n\nr2_score:\n{}'.format(mse_rf_3, r2_rf_3))

MSE:
32952.333916480966

r2_score:
0.8207571338075944


In [ ]:
model_rf = pickle.dumps(rf_3) 
  
# Load the pickled model 
rf_from_pickle = pickle.loads(model_rf) 
  
# Use the loaded pickled model to make predictions 
rf_from_pickle.predict(X_test)

# AUTO_ML

In [11]:
categorical_clms = df.select_dtypes(include=['O']).columns

In [7]:
column_descriptions = {
    'job_performance': 'output',
    'v142':'categorical', 'v180':'categorical', 'reg_tl2':'categorical', 'v166':'categorical',
    'v200':'categorical', 'readytolearn_wle_ca':'categorical', 'v103':'categorical', 'v138':'categorical',
    'iscoskil4':'categorical', 'v19':'categorical', 'nativespeaker':'categorical', 'v155':'categorical',
    'v92':'categorical', 'v251':'categorical', 'writwork_wle_ca':'categorical', 'v178':'categorical', 
    'v126':'categorical',
    'v282':'categorical', 'v57':'categorical', 'v8':'categorical', 'v221':'categorical', 'v15':'categorical',
    'v51':'categorical', 'v132':'categorical', 'v108':'categorical', 'v244':'categorical', 'v153':'categorical',
    'v274':'categorical', 'v289':'categorical', 'v12':'categorical', 'v250':'categorical', 'v233':'categorical',
    'v181':'categorical', 'v101':'categorical', 'writhome_wle_ca':'categorical', 'isic2c':'categorical',
    'v149':'categorical', 'faet12':'categorical', 'v26':'categorical', 'v285':'categorical', 'v13':'categorical',
    'v195':'categorical', 'v106':'categorical', 'edlevel3':'categorical', 'v292':'categorical', 'v91':'categorical',
    'v227':'categorical', 'v27':'categorical', 'isic1c':'categorical', 'v260':'categorical', 'v112':'categorical',
    'ctryrgn':'categorical', 'imyrs_c':'categorical', 'v247':'categorical', 'v93':'categorical', 
    'v216':'categorical', 'v114':'categorical', 'v73':'categorical', 'v237':'categorical', 'v118':'categorical',
    'v25':'categorical', 'v111':'categorical', 'v243':'categorical', 'v50':'categorical', 'v193':'categorical',
    'v141':'categorical', 'v171':'categorical', 'v123':'categorical', 'readhome_wle_ca':'categorical',
    'v28':'categorical', 'v40':'categorical', 'v189':'categorical', 'v157':'categorical', 'v188':'categorical',
    'v58':'categorical', 'fnfe12jr':'categorical', 'taskdisc_wle_ca':'categorical', 'v43':'categorical',
    'influence_wle_ca':'categorical', 'v48':'categorical', 'v69':'categorical', 'v17':'categorical',
    'earnflag':'categorical',
    'v280':'categorical', 'v288':'categorical', 'v158':'categorical', 'v102':'categorical', 'imyrcat':'categorical',
    'v223':'categorical', 'fnfaet12njr':'categorical', 'v263':'categorical', 'v190':'categorical',
    'v47':'categorical', 'v198':'categorical', 'v65':'categorical', 'v246':'categorical', 'v276':'categorical',
    'vet':'categorical', 'v137':'categorical', 'v265':'categorical', 'v261':'categorical', 
    'yearlyincpr':'categorical',
    'v255':'categorical', 'v269':'categorical', 'v284':'categorical', 'v204':'categorical', 'v162':'categorical',
    'v192':'categorical', 'v212':'categorical', 'v165':'categorical', 'gender_r':'categorical', 'v99':'categorical',
    'v151':'categorical', 'v140':'categorical', 'v2':'categorical', 'planning_wle_ca':'categorical',
    'fnfaet12jr':'categorical', 'v229':'categorical', 'v14':'categorical', 'v60':'categorical',
    'ageg5lfs':'categorical',
    'v31':'categorical', 'v191':'categorical', 'learnatwork_wle_ca':'categorical', 'earnmthalldcl':'categorical',
    'v208':'categorical', 'v24':'categorical', 'v291':'categorical', 'v7':'categorical', 'nfe12':'categorical',
    'v218':'categorical', 'v234':'categorical', 'v173':'categorical', 'v203':'categorical', 'v257':'categorical',
    'v42':'categorical', 'v240':'categorical', 'v278':'categorical', 'v96':'categorical', 'v168':'categorical',
    'v277':'categorical', 'v62':'categorical', 'v23':'categorical', 'v85':'categorical', 'v197':'categorical',
    'ageg10lfs':'categorical', 'v275':'categorical', 'v3':'categorical', 'fnfaet12':'categorical',
    'v225':'categorical', 'v34':'categorical', 'v53':'categorical', 'v134':'categorical', 'v175':'categorical',
    'lng_bq':'categorical', 'lng_home':'categorical', 'v271':'categorical', 'v266':'categorical',
    'v36':'categorical', 'v70':'categorical', 'v18':'categorical', 'v186':'categorical', 'v146':'categorical',
    'edcat6':'categorical', 'v214':'categorical', 'v29':'categorical', 'v267':'categorical', 'v139':'categorical',
    'lng_ci':'categorical', 'v222':'categorical', 'icthome_wle_ca':'categorical', 'v59':'categorical',
    'v77':'categorical', 'ictwork_wle_ca':'categorical', 'v177':'categorical', 'v82':'categorical',
    'v248':'categorical', 'v236':'categorical', 'v115':'categorical', 'readwork_wle_ca':'categorical',
    'v230':'categorical', 'monthlyincpr':'categorical', 'v170':'categorical', 'v131':'categorical',
    'v46':'categorical', 'v164':'categorical'
}

In [9]:
df_train, df_test = train_test_split(df, random_state =42, test_size = 0.25)

ml_predictor = Predictor(type_of_estimator='regressor', column_descriptions=column_descriptions)

ml_predictor.train(df_train, model_names='LGBMRegressor',ml_for_analytics=True)

ml_predictor.score(df_test, df_test.job_performance)

Welcome to auto_ml! We're about to go through and make sense of your data using machine learning, and give you a production-ready pipeline to get predictions with.

If you have any issues, or new feature ideas, let us know at http://auto.ml
You are running on version 2.9.10
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'n_estimators': 2000, 'learning_rate': 0.15, 'num_leaves': 8, 'lambda_l2': 0.001, 'histogram_pool_size': 16384}
Running basic data cleaning
Fitting DataFrameVectorizer
Now using the model training_params that you passed in:
{}
After overwriting our defaults with your values, here are the final params that will be used to initialize the model:
{'n_estimators': 2000, 'learning_rate': 0.15, 'num_leaves': 8, 'lambda_l2': 0.001, 'histogram_pool_size': 16384}


***************************************************************************************

[78]	random_holdout_set_from_training_data's rmse: 270.097	random_holdout_set_from_training_data's l2: 72952.5
[79]	random_holdout_set_from_training_data's rmse: 269.487	random_holdout_set_from_training_data's l2: 72623.3
[80]	random_holdout_set_from_training_data's rmse: 269.05	random_holdout_set_from_training_data's l2: 72387.8
[81]	random_holdout_set_from_training_data's rmse: 268.491	random_holdout_set_from_training_data's l2: 72087.3
[82]	random_holdout_set_from_training_data's rmse: 268.049	random_holdout_set_from_training_data's l2: 71850
[83]	random_holdout_set_from_training_data's rmse: 267.79	random_holdout_set_from_training_data's l2: 71711.3
[84]	random_holdout_set_from_training_data's rmse: 267.401	random_holdout_set_from_training_data's l2: 71503.4
[85]	random_holdout_set_from_training_data's rmse: 267.083	random_holdout_set_from_training_data's l2: 71333.2
[86]	random_holdout_set_from_training_data's rmse: 266.942	random_holdout_set_from_training_data's l2: 71257.8
[87]	

[153]	random_holdout_set_from_training_data's rmse: 247.992	random_holdout_set_from_training_data's l2: 61499.8
[154]	random_holdout_set_from_training_data's rmse: 247.946	random_holdout_set_from_training_data's l2: 61477.3
[155]	random_holdout_set_from_training_data's rmse: 247.7	random_holdout_set_from_training_data's l2: 61355.3
[156]	random_holdout_set_from_training_data's rmse: 247.549	random_holdout_set_from_training_data's l2: 61280.3
[157]	random_holdout_set_from_training_data's rmse: 247.377	random_holdout_set_from_training_data's l2: 61195.3
[158]	random_holdout_set_from_training_data's rmse: 247.198	random_holdout_set_from_training_data's l2: 61106.7
[159]	random_holdout_set_from_training_data's rmse: 246.759	random_holdout_set_from_training_data's l2: 60890.2
[160]	random_holdout_set_from_training_data's rmse: 246.738	random_holdout_set_from_training_data's l2: 60879.9
[161]	random_holdout_set_from_training_data's rmse: 246.365	random_holdout_set_from_training_data's l2: 60

[244]	random_holdout_set_from_training_data's rmse: 232.929	random_holdout_set_from_training_data's l2: 54256
[245]	random_holdout_set_from_training_data's rmse: 232.763	random_holdout_set_from_training_data's l2: 54178.6
[246]	random_holdout_set_from_training_data's rmse: 232.666	random_holdout_set_from_training_data's l2: 54133.6
[247]	random_holdout_set_from_training_data's rmse: 232.272	random_holdout_set_from_training_data's l2: 53950.1
[248]	random_holdout_set_from_training_data's rmse: 232.194	random_holdout_set_from_training_data's l2: 53914
[249]	random_holdout_set_from_training_data's rmse: 232.103	random_holdout_set_from_training_data's l2: 53871.7
[250]	random_holdout_set_from_training_data's rmse: 231.971	random_holdout_set_from_training_data's l2: 53810.4
[251]	random_holdout_set_from_training_data's rmse: 231.905	random_holdout_set_from_training_data's l2: 53779.8
[252]	random_holdout_set_from_training_data's rmse: 231.688	random_holdout_set_from_training_data's l2: 5367

[335]	random_holdout_set_from_training_data's rmse: 222.681	random_holdout_set_from_training_data's l2: 49586.6
[336]	random_holdout_set_from_training_data's rmse: 222.521	random_holdout_set_from_training_data's l2: 49515.6
[337]	random_holdout_set_from_training_data's rmse: 222.55	random_holdout_set_from_training_data's l2: 49528.4
[338]	random_holdout_set_from_training_data's rmse: 222.488	random_holdout_set_from_training_data's l2: 49501
[339]	random_holdout_set_from_training_data's rmse: 222.287	random_holdout_set_from_training_data's l2: 49411.5
[340]	random_holdout_set_from_training_data's rmse: 222.177	random_holdout_set_from_training_data's l2: 49362.7
[341]	random_holdout_set_from_training_data's rmse: 221.958	random_holdout_set_from_training_data's l2: 49265.4
[342]	random_holdout_set_from_training_data's rmse: 221.828	random_holdout_set_from_training_data's l2: 49207.5
[343]	random_holdout_set_from_training_data's rmse: 221.8	random_holdout_set_from_training_data's l2: 49195

...

[1806]	random_holdout_set_from_training_data's rmse: 183.701	random_holdout_set_from_training_data's l2: 33746.2
[1807]	random_holdout_set_from_training_data's rmse: 183.701	random_holdout_set_from_training_data's l2: 33746.1
[1808]	random_holdout_set_from_training_data's rmse: 183.7	random_holdout_set_from_training_data's l2: 33745.8
[1809]	random_holdout_set_from_training_data's rmse: 183.689	random_holdout_set_from_training_data's l2: 33741.5
[1810]	random_holdout_set_from_training_data's rmse: 183.683	random_holdout_set_from_training_data's l2: 33739.5
[1811]	random_holdout_set_from_training_data's rmse: 183.667	random_holdout_set_from_training_data's l2: 33733.5
[1812]	random_holdout_set_from_training_data's rmse: 183.677	random_holdout_set_from_training_data's l2: 33737.4
[1813]	random_holdout_set_from_training_data's rmse: 183.669	random_holdout_set_from_training_data's l2: 33734.1
[1814]	random_holdout_set_from_training_data's rmse: 183.664	random_holdout_set_from_training_data

[1879]	random_holdout_set_from_training_data's rmse: 183.518	random_holdout_set_from_training_data's l2: 33678.8
[1880]	random_holdout_set_from_training_data's rmse: 183.51	random_holdout_set_from_training_data's l2: 33676.1
[1881]	random_holdout_set_from_training_data's rmse: 183.526	random_holdout_set_from_training_data's l2: 33681.7
[1882]	random_holdout_set_from_training_data's rmse: 183.505	random_holdout_set_from_training_data's l2: 33674.2
[1883]	random_holdout_set_from_training_data's rmse: 183.528	random_holdout_set_from_training_data's l2: 33682.5
[1884]	random_holdout_set_from_training_data's rmse: 183.503	random_holdout_set_from_training_data's l2: 33673.3
[1885]	random_holdout_set_from_training_data's rmse: 183.496	random_holdout_set_from_training_data's l2: 33670.9
[1886]	random_holdout_set_from_training_data's rmse: 183.492	random_holdout_set_from_training_data's l2: 33669.3
[1887]	random_holdout_set_from_training_data's rmse: 183.481	random_holdout_set_from_training_dat

[1980]	random_holdout_set_from_training_data's rmse: 183.062	random_holdout_set_from_training_data's l2: 33511.7
[1981]	random_holdout_set_from_training_data's rmse: 183.06	random_holdout_set_from_training_data's l2: 33511.1
[1982]	random_holdout_set_from_training_data's rmse: 183.068	random_holdout_set_from_training_data's l2: 33513.8
[1983]	random_holdout_set_from_training_data's rmse: 183.058	random_holdout_set_from_training_data's l2: 33510.2
[1984]	random_holdout_set_from_training_data's rmse: 183.067	random_holdout_set_from_training_data's l2: 33513.4
[1985]	random_holdout_set_from_training_data's rmse: 183.071	random_holdout_set_from_training_data's l2: 33514.9
[1986]	random_holdout_set_from_training_data's rmse: 183.07	random_holdout_set_from_training_data's l2: 33514.7
[1987]	random_holdout_set_from_training_data's rmse: 183.067	random_holdout_set_from_training_data's l2: 33513.3
[1988]	random_holdout_set_from_training_data's rmse: 183.036	random_holdout_set_from_training_data

None


***********************************************
Advanced scoring metrics for the trained regression model on this particular dataset:

Here is the overall RMSE for these predictions:
171.23711624216568

Here is the average of the predictions:
2910.8169337518057

Here is the average actual value on this validation set:
2909.539427410921

Here is the median prediction:
2941.6506317695203

Here is the median actual value:
2940.41165812164

Here is the mean absolute error:
80.02540921300104

Here is the median absolute error (robust to outliers):
12.71419439243914

Here is the explained variance:
0.8369814806895323

Here is the R-squared value:
0.836972406842889
Count of positive differences (prediction > actual):
2522
Count of negative differences:
2478
Average positive difference:
80.59369107243508
Average negative difference:
-79.44703679593341


***********************************************




-171.23711624216568

In [23]:
print("\nMSE (auto_ml):\n{}".format(int((171.23711624216568**2))))


MSE (auto_ml):
29322


In [ ]:
y_pred_aml = ml_predictor.predict(df_test.job_performance)

# Evaluation metrics
mse_rf_3 = mean_squared_error(df_test, df_test.job_performance)
r2_rf_3 = r2_score(df_test, df_test.job_performance)
print('MSE:\n{}\n\nr2_score:\n{}'.format(mse_rf_3, r2_rf_3))

In [25]:
import pickle

In [24]:
model_rf = pickle.dumps(rf_3) 
  
# Load the pickled model 
rf_from_pickle = pickle.loads(model_rf) 
  
# Use the loaded pickled model to make predictions 
rf_from_pickle.predict(X_test)

NameError: name 'pickle' is not defined

In [26]:
df.shape

(20000, 227)

In [27]:
df.columns.values

array(['v142', 'yrsqual', 'v180', 'earnhr', 'reg_tl2', 'v272', 'v166',
       'v200', 'readytolearn_wle_ca', 'v103', 'v138', 'iscoskil4',
       'ictwork', 'v19', 'nativespeaker', 'v155', 'v92', 'v251',
       'planning', 'writwork_wle_ca', 'readhome', 'v178', 'v207', 'v126',
       'v282', 'v57', 'v8', 'v221', 'v15', 'v51', 'v132', 'v108', 'v244',
       'v153', 'v274', 'v252', 'v289', 'v242', 'v12', 'influence', 'v250',
       'yrsget', 'row', 'v233', 'v210', 'v181', 'v101', 'writhome_wle_ca',
       'isic2c', 'writwork', 'learnatwork', 'v149', 'faet12', 'v136',
       'v26', 'v285', 'v13', 'v195', 'v106', 'v33', 'edlevel3', 'v292',
       'earnmth', 'v91', 'nfehrsnjr', 'v227', 'v27', 'isic1c', 'v260',
       'v112', 'ctryrgn', 'imyrs_c', 'v247', 'v93', 'v216', 'v114', 'v73',
       'v237', 'v118', 'v25', 'icthome', 'v111', 'v243', 'readwork',
       'v50', 'v193', 'v141', 'v171', 'v123', 'readhome_wle_ca', 'v28',
       'v40', 'v189', 'v157', 'v188', 'v58', 'fnfe12jr',
       'taskd

In [31]:
df_t_org = pd.read_csv('client-testset-1561457457-252.csv')

In [32]:
df_t = df_t_org[['v142', 'yrsqual', 'v180', 'earnhr', 'reg_tl2', 'v272', 'v166',
       'v200', 'readytolearn_wle_ca', 'v103', 'v138', 'iscoskil4',
       'ictwork', 'v19', 'nativespeaker', 'v155', 'v92', 'v251',
       'planning', 'writwork_wle_ca', 'readhome', 'v178', 'v207', 'v126',
       'v282', 'v57', 'v8', 'v221', 'v15', 'v51', 'v132', 'v108', 'v244',
       'v153', 'v274', 'v252', 'v289', 'v242', 'v12', 'influence', 'v250',
       'yrsget', 'row', 'v233', 'v210', 'v181', 'v101', 'writhome_wle_ca',
       'isic2c', 'writwork', 'learnatwork', 'v149', 'faet12', 'v136',
       'v26', 'v285', 'v13', 'v195', 'v106', 'v33', 'edlevel3', 'v292',
       'earnmth', 'v91', 'nfehrsnjr', 'v227', 'v27', 'isic1c', 'v260',
       'v112', 'ctryrgn', 'imyrs_c', 'v247', 'v93', 'v216', 'v114', 'v73',
       'v237', 'v118', 'v25', 'icthome', 'v111', 'v243', 'readwork',
       'v50', 'v193', 'v141', 'v171', 'v123', 'readhome_wle_ca', 'v28',
       'v40', 'v189', 'v157', 'v188', 'v58', 'fnfe12jr',
       'taskdisc_wle_ca', 'v43', 'influence_wle_ca', 'v48', 'nfehrsjr',
       'v69', 'v17', 'earnflag', 'v280', 'v288', 'v158', 'v102',
       'imyrcat', 'v223', 'fnfaet12njr', 'v263', 'v190', 'v47', 'v198',
       'v65', 'v246', 'v276', 'vet', 'v137', 'v265', 'v261',
       'yearlyincpr', 'v255', 'v269', 'v284', 'v204', 'v162', 'v192',
       'v212', 'v165', 'taskdisc', 'gender_r', 'v99', 'v151', 'v140',
       'v2', 'planning_wle_ca', 'v239', 'fnfaet12jr', 'isco1c', 'v105',
       'v229', 'v14', 'v60', 'ageg5lfs', 'writhome', 'v31', 'v191',
       'learnatwork_wle_ca', 'earnmthalldcl', 'v208', 'v24', 'v291', 'v7',
       'nfe12', 'v218', 'v234', 'v173', 'v203', 'v257', 'v42', 'v240',
       'readytolearn', 'v278', 'v202', 'v96', 'v168', 'v277', 'v62',
       'v23', 'v85', 'v104', 'v133', 'nfehrs', 'v197', 'ageg10lfs',
       'v275', 'v3', 'fnfaet12', 'v225', 'v34', 'v53', 'v134', 'v175',
       'v206', 'lng_bq', 'v224', 'lng_home', 'v271', 'v266', 'v36', 'v70',
       'v18', 'v186', 'v146', 'earnhrppp', 'edcat6', 'v214', 'v29',
       'v267', 'v139', 'lng_ci', 'v222', 'v187', 'icthome_wle_ca', 'v59',
       'leavedu', 'v22', 'v77', 'ictwork_wle_ca', 'v177', 'v82', 'v154',
       'v248', 'v236', 'v135', 'v115', 'readwork_wle_ca', 'v230',
       'monthlyincpr', 'v170', 'v131', 'v46', 'v164']]

In [34]:
# creating 2 diffrent dataframes as numerical and categorical

categorical_clms = df_t.select_dtypes(include=['O']).columns
print("Number of categorical columns: {}".format(len(categorical_clms)))

numerical_clms = df_t.select_dtypes(include=['int64', 'float64']).columns
print("Number of numerical columns: {}".format(len(numerical_clms)))

df_cat = pd.DataFrame(df_t, columns=categorical_clms)
df_num = pd.DataFrame(df_t, columns=numerical_clms)

Number of categorical columns: 186
Number of numerical columns: 40


In [35]:
# replacing outliers with median
for feature in df_num.keys():
    step=1.5*((np.percentile(df_num[feature], q=75))-(np.percentile(df_num[feature], q=25)))
    lower_bound=(np.percentile(df_num[feature], q=25))-step
    upper_bound=(np.percentile(df_num[feature], q=75))+step
    median = round(df_num[feature].median())
    df_num[feature][df_num[feature]<lower_bound] = median
    df_num[feature][df_num[feature]>upper_bound] = median

# filling NaN values with median (because some variables are highly skewed and also some are related with years)
df_num = df_num.fillna(0)

for col in df_num.columns:
    df_num[col]=df_num[col].replace(0,round(df_num[col].median()))

In [36]:
# filling categorical missing with value of "unknown"
df_cat = df_cat.fillna('unknown')

# concatenating both dataframes again
df_t = pd.concat([df_num, df_cat], axis=1, sort=False)

# now we have a %100 filled dataframe
df_t.head(2)

,yrsqual,earnhr,v272,ictwork,planning,readhome,v207,v252,v242,influence,...,v248,v236,v115,readwork_wle_ca,v230,monthlyincpr,v170,v131,v46,v164
0,13.0,0.0,1973.0,0.859661,1.561355,2.528021,2.0,0.0,1.0,2.001258,...,Less than once a week,"Yes, totally",No,More than 40% to 60%,unknown,unknown,To a high extent,Every day,unknown,At least once a week but not every day
1,5.0,0.0,1976.0,0.000000,1.958027,0.499424,0.0,0.0,1.0,1.283328,...,At least once a week,unknown,unknown,More than 20% to 40%,unknown,unknown,To some extent,Every day,One job or business,Every day


In [44]:
rf_3 = RandomForestRegressor(bootstrap=False, max_features=0.15000000000000002, min_samples_leaf=3,
                             min_samples_split=11, n_estimators=100)
rf_3.fit(X_train, y_train)
# df_imp_3 = pd.DataFrame(rf_3.feature_importances_, 
#             index = X_train.columns, columns = ['importance']).sort_values('importance', ascending=False)

# Evaluation
y_pred_rf_3 = rf_3.predict(X_test)

# Evaluation metrics
mse_rf_3 = mean_squared_error(y_test, y_pred_rf_3)
r2_rf_3 = r2_score(y_test, y_pred_rf_3)
print('MSE:\n{}\n\nr2_score:\n{}'.format(mse_rf_3, r2_rf_3))

MSE:
33449.30139013481

r2_score:
0.8180538996570822


In [51]:
df2.shape

(20000, 1287)

In [52]:
df.shape

(20000, 227)

In [53]:
df_t.shape

(20000, 226)

In [54]:
df_t_2.shape

(20000, 1330)

In [56]:
y_pred_aml = ml_predictor.predict(df_t)

In [58]:
len(y_pred_aml)

20000

In [59]:
ml_predictor.save(file_name='auto_ml_saved_pipeline.dill', verbose=True)



We have saved the trained pipeline to a filed called "auto_ml_saved_pipeline.dill"
It is saved in the directory: 
/Users/hy/Documents/05_Upwork/job_performance
To use it to get predictions, please follow the following flow (adjusting for your own uses as necessary:


`from auto_ml.utils_models import load_ml_model
`trained_ml_pipeline = load_ml_model("auto_ml_saved_pipeline.dill")
`trained_ml_pipeline.predict(data)`


Note that this pickle/dill file can only be loaded in an environment with the same modules installed, and running the same Python version.
This version of Python is:
sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)


When passing in new data to get predictions on, columns that were not present (or were not found to be useful) in the training data will be silently ignored.
It is worthwhile to make sure that you feed in all the most useful data points though, to make sure you can get the highest quality predictions.


'/Users/hy/Documents/05_Upwork/job_performance/auto_ml_saved_pipeline.dill'

In [63]:
df_t.to_csv('client-testset_cleaned.csv', sep=',', encoding='utf-8')

In [61]:
df_y_pred_aml = pd.DataFrame(y_pred_aml) 

In [68]:
df_y_pred_aml.to_csv('predictions.csv', sep=',', encoding='utf-8', index=False)

In [65]:
from platform import python_version
print(python_version())

3.7.3


In [ ]:
df = pd.read_csv('client-testset_cleaned.csv')
df = df.drop(['Unnamed: 0'], axis=1)

In [73]:
np.round(y_pred_aml, 5)

array([2498.58991, 1936.37426, 2563.44512, ..., 2573.3893 , 2305.16272,
       3265.37765])

In [67]:
df_y_pred_aml.to_csv('predictions_1.csv', sep=',', encoding='utf-8', index=False)